In [ ]:
import os
from dotenv import load_dotenv

# 필요한 라이브러리 import
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# 환경 변수 로드 및 검증
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다.")


In [ ]:
# PDF 문서 로드
print("PDF 문서를 로드하는 중...")
loader = PyPDFLoader("data/tax_with_table.pdf")
pages = loader.load()
print(f"총 {len(pages)}개의 페이지가 로드되었습니다.")

# 텍스트 분할 (청크 크기 및 중복 크기 최적화)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  
    chunk_overlap=50,  
    separators=["\n\n", "\n", ".", " ", ""]  # 마침표 구분자 추가
)
splits = text_splitter.split_documents(pages)
print(f"총 {len(splits)}개의 청크로 문서가 분할되었습니다.")


In [ ]:
# 임베딩 모델 설정
embeddings_model = OpenAIEmbeddings(
    api_key=OPENAI_API_KEY,
    model="text-embedding-3-small"
)

# 벡터 저장소 생성 - 배치 처리
print("\n벡터 저장소를 생성하는 중...")
batch_size = 50

try:
    first_batch = splits[:batch_size]
    print(f"첫 번째 배치 처리 중... ({len(first_batch)}개 문서)")
    db = Chroma.from_documents(
        first_batch, 
        embeddings_model, 
        persist_directory="./chroma_db"
    )
    print("첫 번째 배치 처리 완료")
    
    # 나머지 배치들을 순차적으로 추가
    for i in range(batch_size, len(splits), batch_size):
        batch = splits[i:i + batch_size]
        batch_num = i // batch_size + 1
        total_batches = len(splits) // batch_size + (1 if len(splits) % batch_size else 0)
        
        print(f"배치 {batch_num}/{total_batches} 처리 중... ({len(batch)}개 문서)")
        db.add_documents(batch)
        print(f"배치 {batch_num} 처리 완료")

except Exception as e:
    print(f"벡터 저장소 생성 중 오류: {str(e)}")
    exit(1)

print("모든 문서의 벡터화가 완료되었습니다!")

In [ ]:
# 질문 설정
query = "비과세소득에 해당하는 소득은 어떤 것들이 있나요? 비과세소득에 대하여 자세히 설명해 주세요."
print(f"\n=== 검색 성능 테스트 ===")
print(f"질문: {query}")

# 1. 다양한 검색 방법으로 테스트
print("\n1. 기본 유사도 검색 (상위 10개):")
docs_basic = db.similarity_search(query, k=10)
for i, doc in enumerate(docs_basic):
    print(f"  {i+1}. {doc.page_content[:100]}...")

# 2. 유사도 점수와 함께 검색하여 성능 확인
print("\n2. 유사도 점수 확인:")
docs_with_scores = db.similarity_search_with_score(query, k=10)
print("점수 분포:")
for i, (doc, score) in enumerate(docs_with_scores):
    print(f"  순위 {i+1}: 점수 {score:.4f}")
    if i < 3:  # 상위 3개만 내용 표시
        print(f"    내용: {doc.page_content[:150]}...")

# 최고 점수와 최저 점수 확인
if docs_with_scores:
    best_score = docs_with_scores[0][1]
    worst_score = docs_with_scores[-1][1]
    print(f"\n점수 범위: {best_score:.4f} ~ {worst_score:.4f}")
    
    # 적절한 임계값 제안
    suggested_threshold = best_score * 0.7  # 최고 점수의 70%
    print(f"제안 임계값: {suggested_threshold:.4f}")

In [ ]:
# 3. 다양한 retriever 설정으로 테스트
retriever_configs = [
    {
        "name": "기본 설정 (상위 4개)",
        "search_type": "similarity",
        "search_kwargs": {"k": 4}
    },
    {
        "name": "더 많은 문서 (상위 8개)",
        "search_type": "similarity", 
        "search_kwargs": {"k": 8}
    },
    {
        "name": "임계값 적용 (0.3)",
        "search_type": "similarity_score_threshold",
        "search_kwargs": {"score_threshold": 0.3, "k": 10}
    },
    {
        "name": "낮은 임계값 (0.1)",
        "search_type": "similarity_score_threshold",
        "search_kwargs": {"score_threshold": 0.1, "k": 10}
    }
]

print("\n3. 다양한 retriever 설정 테스트:")
best_config = None
best_response = None

for config in retriever_configs:
    print(f"\n--- {config['name']} ---")
    
    try:
        retriever = db.as_retriever(
            search_type=config["search_type"],
            search_kwargs=config["search_kwargs"]
        )
        
        # 간단한 프롬프트로 테스트
        template = '''다음 문맥을 바탕으로 질문에 답변해주세요:
        <문맥>
        {context}
        </문맥>

        질문: {input}

        답변:'''
        
        prompt = ChatPromptTemplate.from_template(template)
        
        model = ChatOpenAI(
            model='gpt-3.5-turbo',
            temperature=0,
            api_key=OPENAI_API_KEY
        )
        
        document_chain = create_stuff_documents_chain(model, prompt)
        retrieval_chain = create_retrieval_chain(retriever, document_chain)
        
        response = retrieval_chain.invoke({"input": query})
        
        print(f"검색된 문서 수: {len(response.get('context', []))}")
        print(f"답변 길이: {len(response['answer'])}")
        print(f"답변 미리보기: {response['answer'][:200]}...")
        
        # "찾을 수 없습니다"가 없고 실질적인 답변이 있는 경우를 우선시
        if "찾을 수 없습니다" not in response['answer'] and len(response['answer']) > 50:
            if best_config is None or len(response['answer']) > len(best_response['answer']):
                best_config = config
                best_response = response
                print("현재 최적 설정!")
                
    except Exception as e:
        print(f" 오류 발생: {str(e)}")

In [ ]:
# 최적 설정으로 최종 답변 생성
print(f"\n=== 최종 답변 (최적 설정: {best_config['name'] if best_config else '기본 설정'}) ===")

if best_config:
    final_retriever = db.as_retriever(
        search_type=best_config["search_type"],
        search_kwargs=best_config["search_kwargs"]
    )
else:
    # 기본 설정 사용
    final_retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 8}
    )

# 최종 프롬프트 (더 관대한 조건)
final_template = '''다음 문맥을 참고하여 질문에 답변해주세요. 
문맥에서 직접적인 답변을 찾을 수 없더라도, 관련된 정보가 있다면 그것을 바탕으로 답변해주세요.

<문맥>
{context}
</문맥>

질문: {input}

답변:'''

final_prompt = ChatPromptTemplate.from_template(final_template)
final_document_chain = create_stuff_documents_chain(model, final_prompt)
final_retrieval_chain = create_retrieval_chain(final_retriever, final_document_chain)

final_response = final_retrieval_chain.invoke({"input": query})

print("\n=== 최종 답변 ===")
print(final_response["answer"])

print(f"\n=== 참조된 문서 ({len(final_response.get('context', []))}개) ===")
if final_response.get("context"):
    for i, doc in enumerate(final_response["context"], 1):
        print(f"\n{i}. 문서 내용:")
        print(doc.page_content[:300] + "...")
else:
    print("참조된 문서가 없습니다.")

print("\n 처리가 완료되었습니다.")